# Validation of the optimal solution for LandS problem

Disclaimer: I tried to keep it simple, using standard data structures and straigthforward formulations. Many steps can be done more efficiently. 

To validate the solution, we require and **exact** solver, that is, a solver that use rational numbers, so no rounding error is introduced at any step. In this case, we use QSopt.

In [2]:
import qsoptex
import logging
from fractions import Fraction

#logging.basicConfig(level=logging.DEBUG)


We construct the data of the problem, using fractions instead of float

In [6]:
nscen = 1000000
scen_ex = []
prob_ex = []
for i in range (1,101):
    for j in range(1,101):
        for k in range(1,101):
            scen_ex.append([Fraction(4*(i-1),100),Fraction(4*(j-1),100),Fraction(4*(k-1),100)])
            prob_ex.append(Fraction(1,int(nscen)))
flowEx = [[Fraction(40),Fraction(24),Fraction(4)],
        [Fraction(45),Fraction(27),Fraction(45,10)],
        [Fraction(32),Fraction(192,10),Fraction(32,10)],
        [Fraction(55),Fraction(33),Fraction(55,10)]]
cost_ex = [Fraction(10),Fraction(7),Fraction(16),Fraction(6)]
budget = Fraction(120)
minCap = Fraction(12)
nx = 4
ny = 3


We load the partition of the scenarios from the file. 

In [11]:
partition_new = []
for line in open('partition.txt'):
    listWords = line.split(" ")
    partition_new.append([int(listWords[i]) for i in range(5,len(listWords))])

We compute the average and the probability of each subset of scenarios in the partition

In [15]:
Aggscen_ex = []
probs = []
for subset in partition_new:
    scen = [0,0,0]
    for k in subset:
        for j in range(ny):
            scen[j] += scen_ex[k][j]
    Aggscen_ex.append([Fraction(scen[j],len(subset)) for j in range(ny)])
    probs.append(Fraction(len(subset),nscen))

Now we formulate the problem with the aggregated scenarios. The formulation is the following:
$$[MP] := \min_{x,y \geq 0}   \sum_{i=1}^4  cost_i x_i + \sum_{s\in S} p_s \sum_{i=1}^3 \sum_{j=1}^3flow_{ij}y_{ij}^s$$
$$\sum_{i=1}^4  x_i \geq minCap$$
$$\sum_{i=1}^4 c_ix_i \leq budget$$
$$\sum_{j=1}^3 y_{ij}^s \leq x_i ~\forall~ i\in 1\ldots 4, \forall s\in S$$
$$\sum_{i=1}^4 y_{ij}^s \geq d_j^s  ~\forall~ j\in 1\ldots 3,  \forall s\in S$$

Note that we are aggregating subsets of scenarios, which is equivalent to aggregate the last two constraints, so the solution is a lower bound of the problem.

In [19]:
p = qsoptex.ExactProblem()
for i in range(nx):
    p.add_variable(name='x_'+str(i), objective = cost_ex[i], lower=0)
for k in range(len(partition_new)):
    for i in range(nx):
        for j in range(ny):
            objvar = flowEx[i][j]*probs[k]
            p.add_variable(name='y_'+str(k)+'_'+str(i)+'_'+str(j), objective = objvar, lower=0)
coef = {'x_'+str(i):cost_ex[i] for i in range(nx)}
p.add_linear_constraint(qsoptex.ConstraintSense.LESS, coef, rhs=budget)    
coef = {'x_'+str(i):1 for i in range(nx)}
p.add_linear_constraint(qsoptex.ConstraintSense.GREATER, coef, rhs=minCap)
for k in range(len(partition_new)):
    for i in range(nx):
        coef = {'x_'+str(i):-1}
        for j in range(ny):
            coef['y_'+str(k)+'_'+str(i)+'_'+str(j)] = Fraction(1)
        p.add_linear_constraint(qsoptex.ConstraintSense.LESS, coef, rhs=0)
for k in range(len(partition_new)):
    for j in range(ny):
        coef = {'y_'+str(k)+'_'+str(i)+'_'+str(j):1 for i in range(nx)} 
        p.add_linear_constraint(qsoptex.ConstraintSense.GREATER, coef, rhs=Aggscen_ex[k][j])
p.set_objective_sense(qsoptex.ObjectiveSense.MINIMIZE)






In [22]:
status = p.solve()

We solve and get a feasible solution, and a lower bound of the problem

In [32]:
LB = p.get_objective_value()
[LB,float(LB)]

[Fraction(2256294001, 10000000), 225.6294001]

In [25]:
sol = [p.get_value('x_'+str(i)) for i in range(nx)]
sol

[Fraction(21, 25), Fraction(17, 5), Fraction(47, 25), Fraction(147, 25)]

In [26]:
[float(p.get_value('x_'+str(i))) for i in range(nx)]

[0.84, 3.4, 1.88, 5.88]

Now, we solve the second-stage problem for the solution $x=(0.84,3.4,1.88,5.88)$ for each original scenario.

Note: Python interface of QSopt does not provide the duals variables, so instead we formulate the dual problem, which is:

$$\max_{\nu,\mu \geq 0}  \sum_{j=1}^3 \mu_j d_j^\xi  -   \sum_{i=1}^4 \nu_i \hat{x}_i$$
$$\mu_j - \nu_i \leq f_{ij}  \qquad \forall i\in 1..4, j\in 1..3$$

In [212]:
logging.basicConfig(level=0)


In [27]:
s = qsoptex.ExactProblem()
for i in range(nx):
    s.add_variable(name='v_'+str(i), lower=0)
for j in range(ny):
    s.add_variable(name='u_'+str(j), lower=0)
for i in range(nx):
    for j in range(ny):
        coef = {'u_'+str(j):int(1), 'v_'+str(i):int(-1)}
        s.add_linear_constraint(qsoptex.ConstraintSense.LESS, coef, rhs=flowEx[i][j])
s.set_objective_sense(qsoptex.ObjectiveSense.MAXIMIZE)
s.set_param(qsoptex.Parameter.SIMPLEX_DISPLAY, 0)    

Warning: solving the subproblems can take a long time depending on your computer.

In [30]:
dualsSols = []
dualsObjval = []
for idx in range(len(partition_new)):
    #if (idx%50 == 0 ):
    #    print(idx)
    #it = 0
    for k in partition_new[idx]:
        coef = {'v_'+str(i): -sol[i] for i in range(nx)}
        for j in range(ny):
            coef['u_'+str(j)] = scen_ex[k][j]
        s.set_linear_objective(coef)
        s.solve()
        dual = [s.get_value('u_'+str(j)) for j in range(ny)]
        #if (it > 0):
        #    if (dual != dualsFin[-1:] ):
        #        print("ERROR en idx=%d it=%d" % (idx,it))
        dualsSols.append(dual)
        dualsObjval.append(s.get_objective_value())


And we compute the upper bound, by suming all subproblems objectives and adding the cost of the first-stage solution $x$

In [33]:
UB = sum(dualsObjval)/1000000 + sum([sol[i]*cost_ex[i] for i in range(nx)])
[UB, float(UB)]

[Fraction(2256294001, 10000000), 225.6294001]

In [34]:
[LB, float(LB)]

[Fraction(2256294001, 10000000), 225.6294001]

Since the upper and lower bound are equal, this is the optimal solution of LandS.